In [0]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
from random import shuffle
from google.colab import files, drive

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

Using TensorFlow backend.


In [0]:
# mounting google drive to notebook, gaining data straight from it
# needs the password authorization

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# paths to both classes of image data
condition_path = '/content/drive/My Drive/ColabNotebooks/night_depression_image_data/condition'
control_path = '/content/drive/My Drive/ColabNotebooks/night_depression_image_data/control'

In [0]:
file_list_cont = []
file_list_cond = []

for f in os.listdir(condition_path):
    file_list_cond.append(os.path.join(condition_path, f))

for f in os.listdir(control_path):
    file_list_cont.append(os.path.join(control_path, f))

# counting 20% of each class 
n_test_cond = int(0.2*(len(file_list_cond)))
n_test_cont = int(0.2*(len(file_list_cont)))

random.shuffle(file_list_cond)
random.shuffle(file_list_cont)

print("20% of condition class is " + str(n_test_cond) + " instances")
print("20% of control class is " + str(n_test_cont) + " instances")

20% of condition class is 76 instances
20% of control class is 144 instances


In [0]:
test_data = []

# assesing 20% of each class to test, and the rest to train 
for d in range(n_test_cond):
  test_data.append(file_list_cond[d])
  del file_list_cond[d]

for d in range(n_test_cont):
  test_data.append(file_list_cont[d])
  del file_list_cont[d]
  
train_data = file_list_cond + file_list_cont

print("Test dataset has " + str(len(test_data)) + " of instances.")
print("Train dataset has " + str(len(train_data)) + " of instances.")


Test dataset has 220 of instances.
Train dataset has 886 of instances.


In [0]:
# makes a label ("control"/"condition") to every image after it's name
def one_hot_label(img):
  label = img.split('_')[5]
  if label == 'condition':
    ohl = np.array([1,0])
  elif label == 'control':
    ohl = np.array([0,1])
  return ohl

# load the test image and rescale to force later processing, append to array with the label
def test_data_with_label():
  test_images = []
  for i in test_data:
    path = os.path.join(i)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64,64))
    test_images.append([np.array(img), one_hot_label(i)])
  return test_images

# load the train image and rescale to force later processing, append to array with the label
def train_data_with_label():
  train_images = []
  for i in train_data:
    path = os.path.join(i)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64,64))
    train_images.append([np.array(img), one_hot_label(i)])
  return train_images

In [0]:
# usig functions to asses arrays with the image binary data and the label
training_images = train_data_with_label()
testing_images = test_data_with_label()
tr_img_data = np.array([i[0] for i in training_images]).reshape(-1,64,64,1)
tr_lbl_data = np.array([i[1] for i in training_images])
tst_img_data = np.array([i[0] for i in testing_images]).reshape(-1,64,64,1)
tst_lbl_data = np.array([i[1] for i in testing_images])

In [0]:
model = Sequential()

# LAYERS
# first layer made from ... 
model.add(InputLayer(input_shape=[64,64,1]))
# first convolution, we can adjust the number of kernels & filters
model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same', activation='relu'))
model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same', activation='relu'))
# first poolong
model.add(MaxPool2D(pool_size=5,padding='same'))    

# second layer made from ... 
model.add(Conv2D(filters=64,kernel_size=5,strides=1,padding='same', activation='relu'))
model.add(Conv2D(filters=64,kernel_size=5,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same')) 

# third layer made from ... 
model.add(Conv2D(filters=128,kernel_size=5,strides=1,padding='same', activation='relu'))
model.add(Conv2D(filters=128,kernel_size=5,strides=1,padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same')) 

# INTERPRETATION
# in case of overlearning, drop it out at 0.25 rate
model.add(Dropout(0.25))
# flattening the image between consecutive convolutional layers
model.add(Flatten())
# first neuron description, 512 neurons 
model.add(Dense(512,activation='relu'))
# in case of overlearning, drop it out at 0.5 rate
model.add(Dropout(0.5))
# second neuron description, 2 neurons, 'softmax' to create the probabilities of each of the class o be a class of that instance 
model.add(Dense(2,activation='softmax'))

#LEARNING
# tool for the learning, categorical crossentropy for the binary classification
model.compile(optimizer="adam", loss='categorical_crossentropy',metrics=['accuracy'])

#TRAINING
#epochs - number of learning iterations, 
model.fit(x=tr_img_data, y=tr_lbl_data, epochs=50, batch_size=100)

Epoch 1/50
886/886 [==============================] - 2s 2ms/step - loss: 0.8658 - acc: 0.5914
Epoch 2/50
886/886 [==============================] - 1s 689us/step - loss: 0.6451 - acc: 0.6524
Epoch 3/50
886/886 [==============================] - 1s 685us/step - loss: 0.6425 - acc: 0.6512
Epoch 4/50
886/886 [==============================] - 1s 684us/step - loss: 0.6244 - acc: 0.6580
Epoch 5/50
886/886 [==============================] - 1s 695us/step - loss: 0.6117 - acc: 0.6591
Epoch 6/50
886/886 [==============================] - 1s 690us/step - loss: 0.6215 - acc: 0.6580
Epoch 7/50
886/886 [==============================] - 1s 690us/step - loss: 0.5914 - acc: 0.6648
Epoch 8/50
886/886 [==============================] - 1s 685us/step - loss: 0.5819 - acc: 0.6716
Epoch 9/50
886/886 [==============================] - 1s 684us/step - loss: 0.5569 - acc: 0.6930
Epoch 10/50
886/886 [==============================] - 1s 680us/step - loss: 0.5623 - acc: 0.6761
Epoch 11/50
886/886 [==========

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_76 (Conv2D)           (None, 64, 64, 32)        832       
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 64, 64, 32)        25632     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 13, 13, 64)        51264     
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 13, 13, 64)        102464    
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 3, 3, 128)         204928    
__________

In [0]:
# testing phase
score = model.evaluate(tr_img_data, tr_lbl_data, verbose=0)

# second element is 'accuracy'
score

[0.035844031473642485, 0.9853273137697517]

In [0]:
# gained a better accuracy (from 0.656 to 0.985 by changing to CPU hardware accelerator & adding the second Conv2D to each layer)